In [1]:
import pandas as pd
testing_frame = pd.read_json("/kaggle/input/dactyl-nonadversarial/dactyl_testing_split.json")



In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
CHECKPOINT_PATH = "/kaggle/input/detect-ai-text-deberta-v3-large/pytorch/large/1"
MAX_LEN = 1024
BATCH_SIZE = 16

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    CHECKPOINT_PATH, max_position_embeddings=MAX_LEN
).to("cuda")
pipe = pipeline("text-classification", model=model,tokenizer=tokenizer )

def data():
    for text in testing_frame["text"].values:
        yield text
y_pred = list()
for result in tqdm(pipe(data(),batch_size=16, max_length=MAX_LEN,truncation=True, return_all_scores=True), total=len(testing_frame)):
    for output in result:
        if output["label"] == "LABEL_1":
            y_pred.append(output["score"])



Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
100%|██████████| 91482/91482 [3:42:52<00:00,  6.84it/s]


In [3]:
testing_frame["daigt_pred"] = y_pred
testing_frame[["id","daigt_pred","target","model","domain","category"]].to_csv(f"daigt-results.csv",index=False)
